In [2]:
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END, MessagesState
from langchain_core.messages import HumanMessage, SystemMessage, RemoveMessage
from langgraph.checkpoint.memory import MemorySaver
llm = ChatOpenAI(model = "gpt-4o-mini")

In [3]:
class State(MessagesState):
    summary: str

In [18]:
# Summarize Conversation
def model_call_node(state: State):

    summary = state.get("summary", "")

    if summary:
        system_message = f"Summary of the previous conversation: {summary}"

        messages = [SystemMessage(content=system_message)] + state['messages']

    else:
        messages = state['messages']
        # system_message = "Summarize the conversation on few lines."

    response = llm.invoke(messages)
    return {"messages": response}

In [20]:
def summarize_conversation(state: State):
    
    summary = state.get('summary', '')
    if summary:
        message_summary = (
            f"This is the summary of the previous conversation: {summary}\n\n"
            "Extend the summary by taking into account the new messages above"
        )
    else:
        message_summary = "Create a summary of the recent messgges."

    messages  = state["messages"] + [HumanMessage(content = message_summary)]
    response = llm.invoke(messages)

    delete_messages = [RemoveMessage(m.id) for m in state["messages"][:-2]]
    return {"summary": response.content, "messages": delete_messages}



In [21]:
def should_continue(state: State):
    messages = state["messages"]

    if len(messages)>4:
        return "summarize_conversation"
    
    return END

In [22]:
# Graph
workflow = StateGraph(State)
workflow.add_node("model_call_node", model_call_node)
workflow.add_node("summarize_conversation", summarize_conversation)
 

In [23]:
# Edges
workflow.add_edge(START, "model_call_node")
workflow.add_conditional_edges("model_call_node", should_continue)
workflow.add_edge("summarize_conversation", END)

In [24]:
# Compile
memory = MemorySaver()
graph = workflow.compile(checkpointer=memory)

# Create a thread
config = {"configurable": {"thread_id": "1"}}

In [30]:
input_message = HumanMessage(content="Hi,I am Narcyy")
async for chunk in graph.astream({"messages": [input_message]}, config, stream_mode="updates"):
    print(chunk)

{'model_call_node': {'messages': AIMessage(content='Hello, Narcyy! It seems like you’re introducing yourself again. Is there something specific you’d like to talk about or ask?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 145, 'total_tokens': 174, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None}, id='run-e60b5f52-6647-430d-ba92-ae7e08ba3441-0', usage_metadata={'input_tokens': 145, 'output_tokens': 29, 'total_tokens': 174, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})}}
{'summarize_conversation': {'summary': "Sure! Here’s an extended summary of our conversation:\n\n1. You introduce

In [29]:
input_message = HumanMessage(content="Hi,I am Narcyy")
for chunk in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    print(chunk)

{'messages': [HumanMessage(content='Hi,I am Narcyy', additional_kwargs={}, response_metadata={}, id='95327a99-acb7-4e60-806b-ca31a0e480cf'), AIMessage(content='Hello, Narcyy! It’s nice to meet you. How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 108, 'total_tokens': 129, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None}, id='run-d4e79450-6bf7-4933-9749-938fba6725f6-0', usage_metadata={'input_tokens': 108, 'output_tokens': 21, 'total_tokens': 129, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), HumanMessage(content='Hi,I am Narcyy', additional_kwargs={}, resp